# Inferențe în modele probabilistice: Eliminarea variabilelor
    Tudor Berariu, 2017

Laboratorul precedent a prezentat câteva aspecte teoretice despre rețele Bayesiene. În practică, pentru folosirea lor se conturează câteva probleme:

 - estimarea structurii (a numărului de variabile și a legăturilor directe dintre acestea)
 - estimarea parametrilor (a tabelelor de distribuții de probabilități condiționate)
 - inferențe (calculul unor probabilități oarecare având deja structura și parametrii)
 
În laboratorul curent ne vom ocupa *doar* de problema **inferenței** folosind **algoritmul de eliminare a variabilelor**.

## Inferențe în Rețele Bayesiene

Una dintre problemele esențiale legate de modelele probabilistice este calculul unor întrebări generale de tipul:
$$p\left({\bf Y}|{\bf Z}={\bf z}\right)$$
unde ${\bf Y}$ reprezintă o mulțime de variabile neobservate a căror distribuție de probabilitate este cerută în prezența observațiilor ${\bf Z} = {\bf z}$ (_evidence_).

### Calculul probabilităților condiționate

Rescriind ecuația de mai sus astfel:

$$p\left({\bf Y}|{\bf Z}={\bf z}\right) = \frac{p\left({\bf Y}, {\bf Z}={\bf z}\right)}{p\left({\bf Z}={\bf z}\right)}$$

problema inferenței se reduce la estimarea distribuției comune $P\left({\bf Y}, {\bf Z} = {\bf z}\right)$ și apoi o valoare oarecare din acestă distribuție va putea fi calculată astfel:

$$p\left({\bf Y} = {\bf y} \vert {\bf Z} = {\bf z}\right) = \frac{p\left({\bf Y} = {\bf y}, {\bf Z} = {\bf z}\right)}{\sum_{{\bf y}'}p\left({\bf Y} = {\bf y}', {\bf Z} = {\bf z}\right)}$$

Folosirea acestei expresii permite următoarea relaxare: găsirea unor valori $\phi\left({\bf Y} = {\bf y}, {\bf Z} = {\bf z}\right) \propto P\left({\bf Y} = {\bf y}, {\bf Z} = {\bf z}\right)$, ceea ce permite lucrul cu valori nenormalizate.

$$p\left({\bf Y} = {\bf y} \vert {\bf Z} = {\bf z}\right) = \frac{\phi\left({\bf Y} = {\bf y}, {\bf Z} = {\bf z}\right)}{\sum_{{\bf y}'}\phi\left({\bf Y} = {\bf y}', {\bf Z} = {\bf z}\right)}$$

În cele ce urmează vom descrie un algoritm care lucrează cu astfel de valori, pe care le vom numi factori.

**Extra:** Algoritmul se aplică și rețelelor Markov (varianta _neorientată_ a rețelelor Bayesiene, unde se lucrează cu valori nenormalizate peste clici de variabile)

## Factori

Un factor va fi o tabelă de valori peste o colecție de variabile. De exemplu, pentru valorile `A`, `B` și `C` un factor $\phi_{ABC}$ ar putea arăta așa:

    A | B | C | Value
    --+---+---+------
    0 | 0 | 0 | 0.1
    0 | 0 | 1 | 0.9
    0 | 1 | 0 | 0.8
    0 | 1 | 1 | 0.2
    1 | 0 | 0 | 0.7
    1 | 0 | 1 | 0.4
    1 | 1 | 0 | 0.5
    1 | 1 | 1 | 0.5

unde $\phi_{ABC}\left(A=1, B=0, C=0\right) = 0.7$.
   
### Reprezentarea factorilor

Un factor va fi reprezentat printr-un tuplu cu nume `(vars, values)` unde `vars` este o listă cu numele variabilelor din factorul respectiv, iar `values` este un dicționar ale cărui chei sunt tupluri de valori ale variabilelor, iar valorile dicționarului reprezintă o valoare numerică.

De exemplu, factorul $\phi_{ABC}$ va fi reprezentat astfel:

```(vars=["A", "B", "C"],
    values={(0, 0, 0): .1, (0, 0, 1): .9, (0, 1, 0): .8, (0, 1, 1): .2,
            (1, 0, 0): .7, (1, 0, 1): .4, (1, 1, 0): .5, (1, 1, 1): .5
    }
)```


In [476]:
from collections import namedtuple
Factor = namedtuple("Factor", ["vars", "values"])

def print_factor(phi, indent="\t"):
    line = " | ".join(phi.vars + ["ϕ(" + ",".join(phi.vars) + ")"])
    sep = "".join(["+" if c == "|" else "-" for c in list(line)])
    print(indent + sep)
    print(indent + line)
    print(indent +sep)
    for values, p in phi.values.items():
        print(indent + " | ".join([str(v) for v in values] + [str(p)]))
    print(indent + sep)

    
# Examples

phi_ABC = Factor(vars=["A", "B", "C"],
                 values={(0, 0, 0): .1, (0, 0, 1): .9, (0, 1, 0): .8, (0, 1, 1): .2,
                         (1, 0, 0): .7, (1, 0, 1): .4, (1, 1, 0): .5, (1, 1, 1): .5})
phi_AB = Factor(vars=["A", "B"], values={(0, 0): .1, (0, 1): .9, (1, 0): .8, (1, 1): .2})
phi_BC = Factor(vars=["B", "C"], values={(0, 0): .2, (0, 1): .8, (1, 0): .5, (1, 1): .5})
phi_A = Factor(vars=["A"], values={(0,): .4, (1,): .6})
phi_C = Factor(vars=["C"], values={(0,): .6, (1,): .8})

print_factor(phi_ABC)
print("ϕ(A=1, B=0, C=0) = " + str(phi_ABC.values[(1, 0, 0)]))

	--+---+---+---------
	A | B | C | ϕ(A,B,C)
	--+---+---+---------
	0 | 0 | 0 | 0.1
	0 | 0 | 1 | 0.9
	0 | 1 | 0 | 0.8
	0 | 1 | 1 | 0.2
	1 | 0 | 0 | 0.7
	1 | 0 | 1 | 0.4
	1 | 1 | 0 | 0.5
	1 | 1 | 1 | 0.5
	--+---+---+---------
ϕ(A=1, B=0, C=0) = 0.7


## Multiplicarea a doi factori

Doi factori $\phi_1$ și $\phi_2$ se pot multiplica, obținându-se un nou factor ale cărui valori sunt produse ale valorilor din $\phi_1$ și $\phi_2$. Dacă $\phi_1$ este un factor peste ${\bf X} \cup {\bf Y}$, iar $\phi_2$ este un factor peste ${\bf Y} \cup {\bf Z}$ (${\bf Y}$ reprezintă mulțimea variabilelor comune celor doi factori), atunci:

$$\phi\left(X_1, \ldots\ X_{N_X}, Y_1, \ldots Y_{N_Y}, Z_1, \ldots Z_{N_Z}\right) = \phi_1\left(X_1, \ldots\ X_{N_X}, Y_1, \ldots Y_{N_Y}\right) \cdot \phi_2\left(Y_1, \ldots Y_{N_Y}, Z_1, \ldots Z_{N_Z}\right)$$

De exemplu, fie factorii $\phi_{AB}$ și $\phi_{BC}$:

    --+---+----------          --+---+----------
    A | B | ϕ(A,B)             B | C | ϕ(B,C)
    --+---+----------          --+---+----------
    0 | 0 | 0.100000   <--     0 | 0 | 0.200000
    0 | 1 | 0.900000   !!!     0 | 1 | 0.800000   <--
    1 | 0 | 0.800000           1 | 0 | 0.500000
    1 | 1 | 0.200000           1 | 1 | 0.500000   !!!
    --+---+----------          --+---+----------

Factorul nou se creează combinând toate perechile de intrări din $\phi_{AB}$ și $\phi_{BC}$ pentru care valorile variabilelor comune (în acest caz, $B$) sunt identice.

	--+---+---+---------
	A | B | C | ϕ(A,B,C)
	--+---+---+---------
	0 | 0 | 0 | 0.020000
	0 | 0 | 1 | 0.080000   <--
	0 | 1 | 0 | 0.450000
	0 | 1 | 1 | 0.450000   !!!
	1 | 0 | 0 | 0.160000
	1 | 0 | 1 | 0.640000
	1 | 1 | 0 | 0.100000
	1 | 1 | 1 | 0.100000
	--+---+---+---------


**Cerința 1** : Implementați funcția `multiply` care primește doi factori și întoarce rezultatul înmulțirii celor doi.

In [477]:
# Multiplicarea a doi factori:

def multiply(phi1, phi2):
    assert isinstance(phi1, Factor) and isinstance(phi2, Factor)

    #valori comune celor 2 tabele

    common_values = []
  
    for val1 in phi1[0]:
      for val2 in phi2[0]:
        if val1 == val2 and val1 not in common_values:
          common_values.append(val1)
        
    # print(common_values)

    #valoarea care nu apare in ambele tabele simultan
    
    different_values = []

    for val in phi1[0]:
      if val not in common_values and val not in different_values:
        different_values.append(val)

    for val in phi2[0]:
      if val not in common_values and val not in different_values:
        different_values.append(val)
    
    # print(different_values)

    #toate variabilele
    final_values = common_values + different_values

    # print(final_values)


    index_first_table = {}
    index = 0

    #indecsi prima tabela
    for val in phi1[0]:
      index_first_table[val] = index
      index = index + 1

    index_second_table = {}
    index = 0

    #indecsi a doua tabela
    for val in phi2[0]:
      index_second_table[val] = index
      index = index + 1

    dict_results = {}

    for a in phi1[1]:
      for b in phi2[1]:

        # daca nu sunt variabile comune intre cele 2 tabele
        if common_values == []:
          rez_key = []
          for c in different_values:
            if c in index_first_table:
              rez_key.append(a[index_first_table[c]])
            elif c in index_second_table:
              rez_key.append(b[index_second_table[c]])
          rez = phi1[1][a] * phi2[1][b]
          dict_results[tuple(rez_key)] = rez

        else:
          
          all_commons_equal = True
          for common_value in common_values:
            if a[index_first_table[common_value]] != b[index_second_table[common_value]]:
              all_commons_equal = False
          if all_commons_equal:
            rez = phi1[1][a] * phi2[1][b]              
            rez_key = []
            for common_value in common_values:
              rez_key.append(a[index_first_table[common_value]])
            for c in different_values:
              if c in index_first_table:
                rez_key.append(a[index_first_table[c]])
              elif c in index_second_table:
                rez_key.append(b[index_second_table[c]])
            dict_results[tuple(rez_key)] = rez

    result = Factor(vars = final_values, values = dict_results)

    return result


In [478]:
print_factor(phi_AB)
print("*")
print_factor(phi_BC)
print("=")
print_factor(multiply(phi_AB, phi_BC))

	--+---+-------
	A | B | ϕ(A,B)
	--+---+-------
	0 | 0 | 0.1
	0 | 1 | 0.9
	1 | 0 | 0.8
	1 | 1 | 0.2
	--+---+-------
*
	--+---+-------
	B | C | ϕ(B,C)
	--+---+-------
	0 | 0 | 0.2
	0 | 1 | 0.8
	1 | 0 | 0.5
	1 | 1 | 0.5
	--+---+-------
=
	--+---+---+---------
	B | A | C | ϕ(B,A,C)
	--+---+---+---------
	0 | 0 | 0 | 0.020000000000000004
	0 | 0 | 1 | 0.08000000000000002
	1 | 0 | 0 | 0.45
	1 | 0 | 1 | 0.45
	0 | 1 | 0 | 0.16000000000000003
	0 | 1 | 1 | 0.6400000000000001
	1 | 1 | 0 | 0.1
	1 | 1 | 1 | 0.1
	--+---+---+---------


In [479]:
## Tests for multiply

from itertools import permutations
from operator import mul
from functools import reduce
import sys
from copy import deepcopy

def _check_factor(_phi, all_vars, control):
    assert sorted(_phi.vars) == sorted(all_vars), \
        "Wrong variables: " + ','.join(_phi.vars) + " instead of " + ','.join(all_vars)
    assert len(_phi.values) == 2 ** len(all_vars), \
        "Wrong number of entries in phi.values: " + str(len(_phi.values))
    n = len(all_vars)
    if n > 0:
        for j in range(n + 1):
            vals = [0] * (n - j) + [1] * j
            keys = set([p for p in permutations(vals)])
            p = reduce(mul, [_phi.values[k] for k in keys])
            assert abs(p - control[j]) < 1e-9, \
                "Values for " + str(keys) + " are wrong!"
    else:
        assert abs(_phi.values[()] - control[0]) < 1e-9


def _test_multiply(name1, name2, all_vars, control, verbose=False):
    _phi = eval("multiply(deepcopy(phi_"+name1+"), deepcopy(phi_"+name2+"))")
    if verbose:
        print("Result of ϕ_"+name+" * ϕ_"+name2+":")
        print_factor(_phi)
    sys.stdout.write("Testing  ϕ_"+name1+" * ϕ_"+name2+" ... ")
    _check_factor(_phi, all_vars, control)
    print("OK!!")

_test_multiply("AB", "BC", ["A", "B", "C"], [.02, .00576, .0288, .1], verbose=False)
_test_multiply("A", "BC", ["A", "B", "C"], [.08, .00768, .0288, .3])
_test_multiply("A", "AB", ["A", "B"], [.04, .1728, .12])
_test_multiply("BC", "A", ["C", "A", "B"], [.08, .00768, .0288, .3])
_test_multiply("ABC", "BC", ["C", "A", "B"], [.02, .04032, .008, .25])
_test_multiply("C", "A", ["C", "A"], [.24, .1152, .48])
_test_multiply("A", "C", ["C", "A"], [.24, .1152, .48])
_test_multiply("C", "C", ["C"], [.36, .64])

print("\nMultiply seems ok!\n")

Testing  ϕ_AB * ϕ_BC ... OK!!
Testing  ϕ_A * ϕ_BC ... OK!!
Testing  ϕ_A * ϕ_AB ... OK!!
Testing  ϕ_BC * ϕ_A ... OK!!
Testing  ϕ_ABC * ϕ_BC ... OK!!
Testing  ϕ_C * ϕ_A ... OK!!
Testing  ϕ_A * ϕ_C ... OK!!
Testing  ϕ_C * ϕ_C ... OK!!

Multiply seems ok!



## Eliminarea unei variabile dintr-un factor prin însumare

O variabilă $X_i$ se poate elimina dintr-un factor $\phi$ prin însumarea tuturor valorilor în care celelalte variabile au aceleași valori. Rezultatul este un nou factor $\tau$ al cărui context este dat de toate celelate variabile din $\phi$ în afara lui $X_i$.

Notație: $$\tau \leftarrow \sum_{X_i} \phi$$

$$\tau\left(X_1, \ldots X_{i-1}, X_{i+1}, \ldots, X_N\right) = \sum_{x} \phi \left(X_1, \ldots X_{i-1}, X_i= x, X_{i+1}, \ldots, X_N\right)$$

Prin eliminarea lui $B$ din factorul:
```
--+---+---+----------
A | B | C | ϕ(A,B,C)
--+---+---+----------
0 | 0 | 0 | 0.100000    !!!
0 | 0 | 1 | 0.900000
0 | 1 | 0 | 0.800000    !!!
0 | 1 | 1 | 0.200000
1 | 0 | 0 | 0.700000    <--
1 | 0 | 1 | 0.400000
1 | 1 | 0 | 0.500000    <--
1 | 1 | 1 | 0.500000
--+---+---+----------
```

se obține:

```
--+---+----------
A | C | ϕ(A,C)
--+---+----------
0 | 0 | 0.900000    !!!
0 | 1 | 1.100000
1 | 0 | 1.200000    <---
1 | 1 | 0.900000
--+---+----------
```

**Cerința 2** : Implementați funcția `sum_out` care primește o variabilă `var` și un factor `phi` și întoarce un factor nou obținut prin eliminarea prin însumare a variabilei `var`.

In [480]:
def sum_out(var, phi):
    assert isinstance(phi, Factor) and var in phi.vars

    indexes = {}
    index = 0

    for val in phi[0]:
      indexes[val] = index
      index = index + 1

    var_list = []

    for v in phi[0]:
      if v != var:
        var_list.append(v)

    # print(var_list)

    values_dict = {}

    for a in phi[1]:
      for b in phi[1]:
        list_aux = []
        if a == b:
          continue
        flag = True
        for x in range(len(a)):
          if x == indexes[var]:
            continue
          if a[x] != b[x]:
            flag = False
          list_aux.append(a[x])
        if flag == True:
           values_dict[tuple(list_aux)] = phi[1][a] + phi[1][b]

    result = Factor(vars = var_list, values = values_dict)
    # print(result)
    return result

In [481]:
# Un exemplu

print("Însumând B afară din")
print_factor(phi_ABC)
print("=")
print_factor(sum_out("B", phi_ABC))

Însumând B afară din
	--+---+---+---------
	A | B | C | ϕ(A,B,C)
	--+---+---+---------
	0 | 0 | 0 | 0.1
	0 | 0 | 1 | 0.9
	0 | 1 | 0 | 0.8
	0 | 1 | 1 | 0.2
	1 | 0 | 0 | 0.7
	1 | 0 | 1 | 0.4
	1 | 1 | 0 | 0.5
	1 | 1 | 1 | 0.5
	--+---+---+---------
=
	--+---+-------
	A | C | ϕ(A,C)
	--+---+-------
	0 | 0 | 0.9
	0 | 1 | 1.1
	1 | 0 | 1.2
	1 | 1 | 0.9
	--+---+-------


In [482]:
## Tests for sum_out

def _test_sum_out(var, name, left_vars, control, verbose=False):
    import sys
    from itertools import permutations
    from operator import mul
    from functools import reduce
    _phi = eval("sum_out('"+var+"', phi_"+name+")")
    if verbose:
        print_factor(_phi)
    sys.stdout.write("Testing  sum_"+var+" ϕ_"+name+" ... ")
    _check_factor(_phi, left_vars, control)
    print("OK!!")

_test_sum_out("A", "ABC", ["C", "B"], [.8, 1.69, .7], verbose=False)
_test_sum_out("B", "ABC", ["A", "C"], [.9, 1.32, .9], verbose=False)
_test_sum_out("C", "C", [], [1.4], verbose=False)
_test_sum_out("A", "A", [], [1.], verbose=False)
_test_sum_out("B", "BC", ["C"], [.7, 1.3], verbose=False)

print("\nSummations seems ok!\n")

Testing  sum_A ϕ_ABC ... OK!!
Testing  sum_B ϕ_ABC ... OK!!
Testing  sum_C ϕ_C ... OK!!
Testing  sum_A ϕ_A ... OK!!
Testing  sum_B ϕ_BC ... OK!!

Summations seems ok!



## Eliminarea unei variabile dintr-o mulțime de factori

Dându-se o mulțime de factori $\Phi$, eliminați variabila $X$. Operația se face prin înlocuirea tuturor factorilor care conțin variabila $X$ cu unul obținut prin (1) factorizare și apoi (2) însumare.

`prod_sum(`$\Phi$ `, ` $X$ `)`
 - $\Phi_{X} \leftarrow \left\lbrace \phi \in \Phi \,:\, X \in \phi \right\rbrace$
 - $\omega \leftarrow \prod_{\phi \in \Phi_{X}} \phi$
 - $\tau \leftarrow \sum_{X} \omega$
 - `return` $\Phi \setminus \Phi_{X} \cup \left\lbrace \tau \right\rbrace$

**Cerința 3** : Implementați funcția `prod_sum` care primește o variabilă `var` și o listă de factori și întoarce noua listă de factori obținută prin eliminarea variabilei `var`. Dacă `verbose` este `True`, atunci afișați factorul nou construit (e util mai târziu pentru a urmări pașii algoritmului).

In [483]:
def prod_sum(var, Phi, verbose=False):
    assert isinstance(var, str) and all(
        [isinstance(phi, Factor) for phi in Phi])
    phi_x = []

    #prima bulina
    for p in Phi:
      if var in p[0]:
        phi_x.append(p)
    

    #a doua bulina
    omega = phi_x[0]

    for f in range(1, len(phi_x)):
      omega = multiply(omega, phi_x[f])

    #a treia bulina
    tau = sum_out(var, omega)

    #a patra bulina
    result = []

    for p in Phi:
      if p not in phi_x:
        result.append(p)

    #adaugam si tau in lista
    result.append(tau)

    if verbose==True:
      print(result)

    return result

In [484]:
# Un exemplu
print("Elininând B din :")
print_factor(phi_AB)
print("și")
print_factor(phi_BC)
print("=>")
print_factor(prod_sum("B", [phi_AB, phi_BC])[0])

Elininând B din :
	--+---+-------
	A | B | ϕ(A,B)
	--+---+-------
	0 | 0 | 0.1
	0 | 1 | 0.9
	1 | 0 | 0.8
	1 | 1 | 0.2
	--+---+-------
și
	--+---+-------
	B | C | ϕ(B,C)
	--+---+-------
	0 | 0 | 0.2
	0 | 1 | 0.8
	1 | 0 | 0.5
	1 | 1 | 0.5
	--+---+-------
=>
	--+---+-------
	A | C | ϕ(A,C)
	--+---+-------
	0 | 0 | 0.47000000000000003
	0 | 1 | 0.53
	1 | 0 | 0.26
	1 | 1 | 0.7400000000000001
	--+---+-------


In [485]:
## Test prod_sum

sys.stdout.write("Testing prod-sum (I) ... ")
result = prod_sum("B", [deepcopy(_phi) for _phi in [phi_A, phi_C, phi_ABC, phi_BC]])
assert len(result) == 3
for _phi in result:
    if sorted(_phi.vars) == ["A", "C"]:
        assert abs(_phi.values[(0, 0)] - 0.42) < 1e-9
        assert abs(_phi.values[(0, 1)] * _phi.values[(1, 0)] - 0.3198) < 1e-9
        assert abs(_phi.values[(1, 1)] - 0.57) < 1e-9
    elif sorted(_phi.vars) == ["A"]:
        assert abs(_phi.values[(0,)] - 0.4) < 1e-9
        assert abs(_phi.values[(1,)] - 0.6) < 1e-9
    elif sorted(_phi.vars) == ["C"]:
        assert abs(_phi.values[(0,)] - 0.6) < 1e-9
        assert abs(_phi.values[(1,)] - 0.8) < 1e-9
print("OK!")

sys.stdout.write("Testing prod-sum (II) ... ")
result = prod_sum("A", [deepcopy(_phi) for _phi in [phi_A, phi_C, phi_ABC, phi_BC]])
assert len(result) == 3
for _phi in result:
    if sorted(_phi.vars) == ["B", "C"]:
        assert abs(_phi.values[(0, 0)] - 0.2) < 1e-9 or abs(_phi.values[(0, 0)] - 0.46) < 1e-9
        assert abs(_phi.values[(0, 1)] * _phi.values[(1, 0)] - 0.4) < 1e-9 or \
               abs(_phi.values[(0, 1)] * _phi.values[(1, 0)] - 0.372) < 1e-9
        assert abs(_phi.values[(1, 1)] - 0.5) < 1e-9 or abs(_phi.values[(1, 1)] - 0.38) < 1e-9
    elif sorted(_phi.vars) == ["C"]:
        assert abs(_phi.values[(0,)] - 0.6) < 1e-9
        assert abs(_phi.values[(1,)] - 0.8) < 1e-9
print("OK!")
print("Prod-Sum seems ok!")

Testing prod-sum (I) ... OK!
Testing prod-sum (II) ... OK!
Prod-Sum seems ok!


## Eliminarea variabilelor

Dându-se o mulțime de factori $\Phi$ și o mulțime de variabile de eliminat $\bf{Z}$, să se construiască factorul obținut după eliminarea tuturor variabilelor $Z_i$.

`variable_elimination(` $\Phi$ `,` ${\bf Z}$ `)`
 - `for` $Z_i \in {\bf Z}$:
   - $\Phi \leftarrow $ `prod_sum(` $Z_i$ `,` $\Phi$ `)`
 - `return` $\prod_{\phi \in \Phi} \phi$
 
Ordinea în care se iau variabilele din ${\bf Z}$ poate infleunța eficiența algoritmului. (Vezi BONUS.)

** Cerința 4 ** : Implementați funcția `variable_elimination`. Aceasta trebuie să întoarcă un singur factor. Folosiți argumentul `verbose` și în apelurile funcției `prod_sum`.

In [486]:
def variable_elimination(Phi, Z, verbose=False):

    phi = deepcopy(Phi)

    for z_i in Z:
      phi = prod_sum(z_i, phi, verbose)

    result = phi[0]

    for p in range(1,len(phi)):
      result = multiply(result, phi[p])

    return result

In [487]:
## Testing Variable elimination

def _test_variable_elimination(Phi, Vars, left_vars, control, verbose=False):

    
    var_list = '["' + '", "'.join(Vars) + '"]'
    factor_list = '[' + ','.join([("deepcopy(phi_"+name + ")") for name in Phi]) +']'
    name_list = '[' + ','.join([("ϕ_"+name) for name in Phi]) +']'
    _phi = eval("variable_elimination("+factor_list+", "+var_list+")")
    if verbose:
        print_factor(_phi)
    sys.stdout.write("Testing  eliminate_var "+var_list+" from "+name_list+" ... ")
    _check_factor(_phi, left_vars, control)
    print("OK!!")

_test_variable_elimination(["A", "C"], ["C"], ["A"], [0.56, 0.84])
_test_variable_elimination(["ABC", "BC", "AB", "A"], ["C", "B"], ["A"], [0.2096, 0.2808])
_test_variable_elimination(["ABC", "BC", "AB", "A"], ["C", "B", "A"], [], [0.4904])
_test_variable_elimination(["ABC", "AB", "BC", "A"], ["A", "B", "C"], [], [0.4904])
_test_variable_elimination(["ABC"], ["A", "B", "C"], [], [4.1])


Testing  eliminate_var ["C"] from [ϕ_A,ϕ_C] ... OK!!
Testing  eliminate_var ["C", "B"] from [ϕ_ABC,ϕ_BC,ϕ_AB,ϕ_A] ... OK!!
Testing  eliminate_var ["C", "B", "A"] from [ϕ_ABC,ϕ_BC,ϕ_AB,ϕ_A] ... OK!!
Testing  eliminate_var ["A", "B", "C"] from [ϕ_ABC,ϕ_AB,ϕ_BC,ϕ_A] ... OK!!
Testing  eliminate_var ["A", "B", "C"] from [ϕ_ABC] ... OK!!


## Reducerea factorilor conform observațiilor

**Cerința 5** : Reduceți factorii eliminanând intrările ce nu corespund observațiilor făcute. Implementați funcția `coniditon_factors` care primește o listă de factori `Phi`, un dicțonar de observații și întoarce o listă nouă în care factorii ce conțin variabile din `Z` sunt reduși la liniile conforme cu observațiile făcute.

In [488]:
def condition_factors(Phi, Z, verbose=False):

    phi = deepcopy(Phi)

    for z_i in Z:
      for p in phi:
        if z_i in p[0]:
          useless_lines = []
          idx = p[0].index(z_i)
          for a in p[1]:
            if a[idx] != Z[z_i]:
              useless_lines.append(a)
          for useless_line in useless_lines:
            del p[1][useless_line]


    if verbose == True:
      print(phi)
    return phi

In [489]:
# Un exemplu
print("Aplicand B=0 in factorul")
print_factor(phi_ABC)
print("=>")
print_factor(condition_factors([phi_ABC], {"B": 0})[0])

Aplicand B=0 in factorul
	--+---+---+---------
	A | B | C | ϕ(A,B,C)
	--+---+---+---------
	0 | 0 | 0 | 0.1
	0 | 0 | 1 | 0.9
	0 | 1 | 0 | 0.8
	0 | 1 | 1 | 0.2
	1 | 0 | 0 | 0.7
	1 | 0 | 1 | 0.4
	1 | 1 | 0 | 0.5
	1 | 1 | 1 | 0.5
	--+---+---+---------
=>
	--+---+---+---------
	A | B | C | ϕ(A,B,C)
	--+---+---+---------
	0 | 0 | 0 | 0.1
	0 | 0 | 1 | 0.9
	1 | 0 | 0 | 0.7
	1 | 0 | 1 | 0.4
	--+---+---+---------


In [490]:
# Teste pentru condition_factors

phi_ABC = Factor(vars=["A", "B", "C"],
                 values={(0, 0, 0): .1, (0, 0, 1): .9, (0, 1, 0): .8, (0, 1, 1): .2,
                         (1, 0, 0): .7, (1, 0, 1): .4, (1, 1, 0): .5, (1, 1, 1): .5})

_phi = condition_factors([phi_ABC], {"B": 0})[0]
assert sorted(_phi.vars) == ["A", "B", "C"]
assert len(_phi.values) == 4 and abs(_phi.values[(0, 0, 0)] - .1) < 1e-7
_phi = condition_factors([phi_ABC], {"B": 0, "A": 1})[0]
assert sorted(_phi.vars) == ["A", "B", "C"] and len(_phi.values) == 2
print("Condition factors seems ok!")

Condition factors seems ok!


## Realizarea inferențelor în Rețele Bayesiene

$$P\left({\bf Y} \vert {\bf Z} = {\bf z}\right) = \frac{P\left({\bf Y}, {\bf Z} = {\bf z}\right)}{P\left(\bf{Z = z}\right)}$$

Realizarea inferențelor de tipul generic de mai sus se face în următorii pași:

 - tabelele cu distribuțiile condiționate sunt transformate în factori
 - factorii ce conțin variabile din ${\bf Z}$ sunt reduși la liniile care respectă ${\bf Z} = {\bf z}$
 - fie $\Phi$ mulțimea factorilor astfel obținuți
 - fie $\phi_{YZ}$ factorul obținut prin eliminarea tuturor celorlalte variabile:

     * $\phi_{YZ} \leftarrow $ `var_elimination` $\left(\Phi, {\bf X} \setminus \left({\bf Y} \cup {\bf Z}\right)\right)$
 - atunci $$P({\bf Y} = {\bf y}| {\bf Z}= {\bf z}) = \frac{\phi_{YZ}({\bf Y}={\bf y})}{\sum_{{\bf Y}} \phi_{YZ}}$$


In [491]:
from random import shuffle

def query(X, Y, Z, Phi, Other=None, verbose=False):
    """
    X - full list of variables
    Y - query variables
    Z - dictionary with observations
    Phi - the list with all factor
    Ohter - an order over variables in X \ (Y U Z); None to pick a random one
    verbose - display factors as they are created
    """

    if verbose:
        print("\n-------------\nInitial factors:")
        for phi in Phi:
            print_factor(phi)

    Phi = condition_factors(Phi, Z, verbose=verbose)  # Condition factors on Z=z

    if Other is None:
        Other = [x for x in X if (x not in Y and x not in Z)]  # Variables that need to be eliminated
        shuffle(Other)
    else:
        assert sorted(Other) == sorted([x for x in X if (x not in Y and x not in Z)])
    if verbose:
        print("\n-------------\nEliminating variables in the following order: " + ",".join(Other))

    phi = variable_elimination(Phi, Other, verbose=verbose)  # Eliminate other variables then Y and Z
    
    # Normalize factor to represent the conditional probability p(Y|Z=z)
    s = sum(phi.values.values())
    prob = Factor(vars=phi.vars, values={k: v / s for (k, v) in phi.values.items()})
    print("\n-----------------\nProbabilitatea ceruta:")
    print_factor(prob)


## Exemplu

Urmăriți exemplul din PDF-ul atașat.

In [492]:
phi_a = Factor(vars=["A"], values={(0,): .7, (1,): .3})
phi_b = Factor(vars=["B"], values={(0,): .5, (1,): .5})
phi_c = Factor(vars=["C"], values={(0,): .4, (1,): .6})

phi_d = Factor(vars=["A", "B", "D"],
               values={(0, 0, 0): .75, (0, 0, 1): .25, (0, 1, 0): .7, (0, 1, 1): .3,
                       (1, 0, 0): .6, (1, 0, 1): .4, (1, 1, 0): .2, (1, 1, 1): .8
                      })
phi_e = Factor(vars=["C", "E"],
               values={(0, 0): .25, (0, 1): .75, (1, 0): .75, (1, 1): .25})

phi_f = Factor(vars=["A", "D", "F"],
               values={(0, 0, 0): .6, (0, 0, 1): .4, (0, 1, 0): .4, (0, 1, 1): .6,
                       (1, 0, 0): .7, (1, 0, 1): .3, (1, 1, 0): .8, (1, 1, 1): .2
                      })
phi_g = Factor(vars=["D", "E", "G"],
               values={(0, 0, 0): .1, (0, 0, 1): .9, (0, 1, 0): .2, (0, 1, 1): .8,
                       (1, 0, 0): .5, (1, 0, 1): .5, (1, 1, 0): .4, (1, 1, 1): .6
                      })

all_vars = ["A", "B", "C", "D", "E", "F", "G"]
Phi = [phi_a, phi_b, phi_c, phi_d, phi_e, phi_f, phi_g]

In [493]:
# Algoritmul ar trebui să ajungă la probabilitățile din tabele

# Verificati ca algoritmul "ajunge" corect la valorile din tabele
query(all_vars, ["F"], {"A": 0, "D": 1}, Phi)
query(all_vars, ["G"], {"D": 0, "E": 1}, Phi)


-----------------
Probabilitatea ceruta:
	--+---+---+---------
	D | A | F | ϕ(D,A,F)
	--+---+---+---------
	1 | 0 | 0 | 0.39999999999999997
	1 | 0 | 1 | 0.6
	--+---+---+---------

-----------------
Probabilitatea ceruta:
	--+---+---+---------
	D | E | G | ϕ(D,E,G)
	--+---+---+---------
	0 | 1 | 0 | 0.2
	0 | 1 | 1 | 0.8
	--+---+---+---------


In [494]:
# Exemplul din PDF-ul atașat

query(all_vars, ["C", "F"], {"G": 0}, Phi, Other=["E", "B", "A", "D"], verbose=True)



-------------
Initial factors:
	--+-----
	A | ϕ(A)
	--+-----
	0 | 0.7
	1 | 0.3
	--+-----
	--+-----
	B | ϕ(B)
	--+-----
	0 | 0.5
	1 | 0.5
	--+-----
	--+-----
	C | ϕ(C)
	--+-----
	0 | 0.4
	1 | 0.6
	--+-----
	--+---+---+---------
	A | B | D | ϕ(A,B,D)
	--+---+---+---------
	0 | 0 | 0 | 0.75
	0 | 0 | 1 | 0.25
	0 | 1 | 0 | 0.7
	0 | 1 | 1 | 0.3
	1 | 0 | 0 | 0.6
	1 | 0 | 1 | 0.4
	1 | 1 | 0 | 0.2
	1 | 1 | 1 | 0.8
	--+---+---+---------
	--+---+-------
	C | E | ϕ(C,E)
	--+---+-------
	0 | 0 | 0.25
	0 | 1 | 0.75
	1 | 0 | 0.75
	1 | 1 | 0.25
	--+---+-------
	--+---+---+---------
	A | D | F | ϕ(A,D,F)
	--+---+---+---------
	0 | 0 | 0 | 0.6
	0 | 0 | 1 | 0.4
	0 | 1 | 0 | 0.4
	0 | 1 | 1 | 0.6
	1 | 0 | 0 | 0.7
	1 | 0 | 1 | 0.3
	1 | 1 | 0 | 0.8
	1 | 1 | 1 | 0.2
	--+---+---+---------
	--+---+---+---------
	D | E | G | ϕ(D,E,G)
	--+---+---+---------
	0 | 0 | 0 | 0.1
	0 | 0 | 1 | 0.9
	0 | 1 | 0 | 0.2
	0 | 1 | 1 | 0.8
	1 | 0 | 0 | 0.5
	1 | 0 | 1 | 0.5
	1 | 1 | 0 | 0.4
	1 | 1 | 1 | 0.6
	--+---+---+---------
